In [1]:
import os
import shap

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

/Users/aaron/Desktop/HW/MED_264/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# get data from all 3 datasets
full_feature_data = pd.DataFrame()
full_ema_data = pd.DataFrame()
full_bfi_data = pd.DataFrame()

for i in range(2, 5):
    curr_folder = './data/globem/INS-W_'+str(i)
    rapids_df = pd.read_csv(curr_folder+'/FeatureData/rapids.csv', usecols=lambda col: col.endswith('14dhist') and 'norm' in col or col=='pid' or col=='date')
    full_feature_data = pd.concat([full_feature_data, rapids_df], ignore_index=True)
    
    ema_df = pd.read_csv(curr_folder+'/SurveyData/ema.csv')
    full_ema_data = pd.concat([full_ema_data, ema_df], ignore_index=True)
    
    bfi_df = pd.read_csv(curr_folder+'/SurveyData/pre.csv')[['pid', 'date', \
                                    'BFI10_extroversion_PRE', \
                                    'BFI10_agreeableness_PRE', \
                                    'BFI10_conscientiousness_PRE', \
                                    'BFI10_neuroticism_PRE', \
                                    'BFI10_openness_PRE']]
    full_bfi_data = pd.concat([full_bfi_data, bfi_df], ignore_index=True)
print(len(full_feature_data))
full_feature_data.to_csv('merged_unfiltered_feature_data.csv')
full_ema_data.to_csv('merged_unfiltered_ema_data.csv')
full_bfi_data.to_csv('merged_unfiltered_bfi_data.csv')

55342


In [3]:
# merge wrt pid and date
ema_data = pd.read_csv('merged_unfiltered_ema_data.csv')[['pid', 'date', \
                                    'phq4_EMA', 'phq4_anxiety_EMA', 'phq4_depression_EMA']]
clean_ema_data = ema_data.dropna()
bfi_data = pd.read_csv('merged_unfiltered_bfi_data.csv')[['pid', \
                                    'BFI10_extroversion_PRE', \
                                    'BFI10_agreeableness_PRE', \
                                    'BFI10_conscientiousness_PRE', \
                                    'BFI10_neuroticism_PRE', \
                                    'BFI10_openness_PRE']]
clean_bfi_data = bfi_data.dropna()
merged_data = pd.merge(full_feature_data, clean_ema_data, on=['pid', 'date'], how='inner')
merged_data = pd.merge(merged_data, clean_bfi_data, on=['pid'], how='inner')
print(len(merged_data.columns))
print(len(merged_data))

216
5257


In [4]:
# no longer need pid and date
merged_data.drop(columns=['pid', 'date'], inplace=True)
print(len(merged_data.columns))

214


In [5]:
# drop string columns, can come back to this and one hot encode these but doesnt seem worth it
for col in merged_data.columns:
    if merged_data[col].apply(lambda x: isinstance(x, str)).any():
        merged_data.drop(columns=col, inplace=True)
        continue
print(len(merged_data.columns))

213


In [6]:
# impute wrt nn for NaN data
numeric_data = merged_data.select_dtypes(include=['number'])
imputer = KNNImputer(n_neighbors=1)
imputed_data = imputer.fit_transform(merged_data)
merged_data[numeric_data.columns] = imputed_data

In [7]:
# normalize bfi data
columns_to_normalize = [
    'BFI10_extroversion_PRE', 
    'BFI10_agreeableness_PRE', 
    'BFI10_conscientiousness_PRE', 
    'BFI10_neuroticism_PRE', 
    'BFI10_openness_PRE'
]
scaler = MinMaxScaler()
merged_data[columns_to_normalize] = scaler.fit_transform(merged_data[columns_to_normalize])

In [8]:
# make sure no na at this point
merged_data.isna().any().any()

np.False_

In [9]:
merged_data.to_csv('merged_knn_imputed_filtered_data_phq.csv')

In [10]:
# split into our feature and target variables
phq = merged_data['phq4_EMA']
anx_phq = merged_data['phq4_anxiety_EMA']
dep_phq = merged_data['phq4_depression_EMA']
merged_data = merged_data.drop(columns=['phq4_EMA', 'phq4_anxiety_EMA', 'phq4_depression_EMA'])
merged_data_no_bfi = merged_data.drop(columns=['BFI10_extroversion_PRE', 'BFI10_agreeableness_PRE', 'BFI10_conscientiousness_PRE', 'BFI10_neuroticism_PRE', 'BFI10_openness_PRE'])

In [11]:
# get statisticallt relevant for phq no bfi and w/bfi
gamma_model = sm.GLM(phq, merged_data)
gamma_results = gamma_model.fit()
p_values = gamma_results.pvalues
significant_columns = p_values[p_values < 0.5].index
new_merged_data_phq = merged_data[significant_columns]

gamma_model = sm.GLM(phq, merged_data_no_bfi)
gamma_results = gamma_model.fit()
p_values = gamma_results.pvalues
significant_columns = p_values[p_values < 0.5].index
new_merged_data_no_bfi_phq = merged_data_no_bfi[significant_columns]

In [12]:
# get columns in one but not the other
columns_in_df1_not_in_df2 = set(new_merged_data_no_bfi_phq.columns) - set(new_merged_data_phq.columns)

columns_in_df2_not_in_df1 = set(new_merged_data_phq.columns) - set(new_merged_data_no_bfi_phq.columns)

print("Columns only in not BFI predicting phq:")
for col in columns_in_df1_not_in_df2:
    print(f"  - {col}")

print('---------')
print("Columns only in BFI predicting phq:")
for col in columns_in_df2_not_in_df1:
    print(f"  - {col}")

Columns only in not BFI predicting phq:
  - f_screen:phone_screen_rapids_avgdurationunlock_locmap_exercise_norm:14dhist
  - f_slp:fitbit_sleep_intraday_rapids_mediandurationasleepunifiedmain_norm:14dhist
  - f_screen:phone_screen_rapids_mindurationunlock_locmap_living_norm:14dhist
  - f_call:phone_calls_rapids_outgoing_entropyduration_norm:14dhist
  - f_steps:fitbit_steps_summary_rapids_maxsumsteps_norm:14dhist
  - f_loc:phone_locations_locmap_duration_in_locmap_greens_norm:14dhist
  - f_screen:phone_screen_rapids_sumdurationunlock_norm:14dhist
  - f_steps:fitbit_steps_intraday_rapids_mindurationactivebout_norm:14dhist
  - f_loc:phone_locations_barnett_avgflightdur_norm:14dhist
  - f_blue:phone_bluetooth_doryab_stdscansothers_norm:14dhist
  - f_blue:phone_bluetooth_doryab_stdscansown_norm:14dhist
  - f_screen:phone_screen_rapids_mindurationunlock_locmap_study_norm:14dhist
  - f_call:phone_calls_rapids_incoming_stdduration_norm:14dhist
  - f_slp:fitbit_sleep_intraday_rapids_mindurationa

In [13]:
# get statisticallt relevant for anxiety phq no bfi and w/bfi
gamma_model = sm.GLM(anx_phq, merged_data)
gamma_results = gamma_model.fit()
p_values = gamma_results.pvalues
significant_columns = p_values[p_values < 0.5].index
new_merged_data_anx_phq = merged_data[significant_columns]

gamma_model = sm.GLM(anx_phq, merged_data_no_bfi)
gamma_results = gamma_model.fit()
p_values = gamma_results.pvalues
significant_columns = p_values[p_values < 0.5].index
new_merged_data_no_bfi_anx_phq = merged_data_no_bfi[significant_columns]

In [14]:
# get columns in one but not the other
columns_in_df1_not_in_df2 = set(new_merged_data_no_bfi_anx_phq.columns) - set(new_merged_data_anx_phq.columns)

columns_in_df2_not_in_df1 = set(new_merged_data_anx_phq.columns) - set(new_merged_data_no_bfi_anx_phq.columns)

print("Columns only in not BFI predicting anxiety phq:")
for col in columns_in_df1_not_in_df2:
    print(f"  - {col}")

print('---------')
print("Columns only in BFI predicting anxiety phq:")
for col in columns_in_df2_not_in_df1:
    print(f"  - {col}")

Columns only in not BFI predicting anxiety phq:
  - f_slp:fitbit_sleep_intraday_rapids_sumdurationasleepunifiedmain_norm:14dhist
  - f_loc:phone_locations_barnett_avgflightlen_norm:14dhist
  - f_loc:phone_locations_barnett_wkenddayrtn_norm:14dhist
  - f_blue:phone_bluetooth_doryab_uniquedevicesown_norm:14dhist
  - f_screen:phone_screen_rapids_mindurationunlock_locmap_living_norm:14dhist
  - f_loc:phone_locations_doryab_avglengthstayatclusters_norm:14dhist
  - f_steps:fitbit_steps_intraday_rapids_sumsteps_norm:14dhist
  - f_call:phone_calls_rapids_outgoing_entropyduration_norm:14dhist
  - f_loc:phone_locations_locmap_duration_in_locmap_greens_norm:14dhist
  - f_screen:phone_screen_rapids_sumdurationunlock_norm:14dhist
  - f_steps:fitbit_steps_intraday_rapids_mindurationactivebout_norm:14dhist
  - f_loc:phone_locations_barnett_avgflightdur_norm:14dhist
  - f_call:phone_calls_rapids_incoming_count_norm:14dhist
  - f_blue:phone_bluetooth_doryab_stdscansown_norm:14dhist
  - f_blue:phone_blu

In [15]:
# get statisticallt relevant for depression phq no bfi and w/bfi
gamma_model = sm.GLM(dep_phq, merged_data)
gamma_results = gamma_model.fit()
p_values = gamma_results.pvalues
significant_columns = p_values[p_values < 0.5].index
new_merged_data_dep_phq = merged_data[significant_columns]

gamma_model = sm.GLM(dep_phq, merged_data_no_bfi)
gamma_results = gamma_model.fit()
p_values = gamma_results.pvalues
significant_columns = p_values[p_values < 0.5].index
new_merged_data_no_bfi_dep_phq = merged_data_no_bfi[significant_columns]

In [16]:
# get columns in one but not the other
columns_in_df1_not_in_df2 = set(new_merged_data_no_bfi_dep_phq.columns) - set(new_merged_data_dep_phq.columns)

columns_in_df2_not_in_df1 = set(new_merged_data_dep_phq.columns) - set(new_merged_data_no_bfi_dep_phq.columns)

print("Columns only in not BFI predicting depression phq:")
for col in columns_in_df1_not_in_df2:
    print(f"  - {col}")

print('---------')
print("Columns only in BFI predicting depression phq:")
for col in columns_in_df2_not_in_df1:
    print(f"  - {col}")

Columns only in not BFI predicting depression phq:
  - f_slp:fitbit_sleep_intraday_rapids_mediandurationasleepunifiedmain_norm:14dhist
  - f_screen:phone_screen_rapids_mindurationunlock_locmap_living_norm:14dhist
  - f_steps:fitbit_steps_intraday_rapids_sumsteps_norm:14dhist
  - f_call:phone_calls_rapids_outgoing_entropyduration_norm:14dhist
  - f_loc:phone_locations_locmap_duration_in_locmap_greens_norm:14dhist
  - f_blue:phone_bluetooth_doryab_countscansleastfrequentdevicewithinsegmentsall_norm:14dhist
  - f_call:phone_calls_rapids_outgoing_meanduration_norm:14dhist
  - f_screen:phone_screen_rapids_sumdurationunlock_norm:14dhist
  - f_loc:phone_locations_locmap_duration_in_locmap_exercise_norm:14dhist
  - f_call:phone_calls_rapids_incoming_stdduration_norm:14dhist
  - f_loc:phone_locations_doryab_numberlocationtransitions_norm:14dhist
  - f_call:phone_calls_rapids_incoming_modeduration_norm:14dhist
  - f_slp:fitbit_sleep_summary_rapids_lastbedtimemain_norm:14dhist
  - f_screen:phone_

In [17]:
# run a gridsearch on all variances of data + target variable combos
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [18]:
# sanity check baseline
print(mean_squared_error(np.mean(phq)*np.ones_like(phq), phq))

8.084497823838712


In [19]:
# no bfi phq
rf = RandomForestRegressor()
param_grid = {
    'n_estimators': [5, 10, 15, 20, 25, 30, 35, 40],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=mse_scorer,
    cv=kf,
    verbose=1,
    n_jobs=-1  
)
grid_search.fit(new_merged_data_no_bfi_phq, phq)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated score:", grid_search.best_score_)

best_params = grid_search.best_params_
final_rf_model = RandomForestRegressor(**best_params)
final_rf_model.fit(new_merged_data_no_bfi_phq, phq)
final_cv_scores = cross_val_score(
    final_rf_model, 
    new_merged_data_no_bfi_phq, 
    phq, 
    cv=kf, 
    scoring=mse_scorer
)

print("Final model cross-validated scores:", final_cv_scores)
print("Final model mean score:", np.mean(final_cv_scores))

Fitting 5 folds for each of 360 candidates, totalling 1800 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 35}
Best cross-validated score: -6.276123051567764
Final model cross-validated scores: [-6.25303362 -6.18356037 -6.38514592 -6.2570679  -6.81847309]
Final model mean score: -6.379456181298551


In [20]:
# with bfi phq
rf = RandomForestRegressor()
param_grid = {
    'n_estimators': [5, 10, 15, 20, 25, 30, 35, 40],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=mse_scorer,
    cv=kf,
    verbose=1,
    n_jobs=-1  
)
grid_search.fit(new_merged_data_phq, phq)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated score:", grid_search.best_score_)

best_params = grid_search.best_params_
final_rf_model = RandomForestRegressor(**best_params)
final_rf_model.fit(new_merged_data_phq, phq)
final_cv_scores = cross_val_score(
    final_rf_model, 
    new_merged_data_phq, 
    phq, 
    cv=kf, 
    scoring=mse_scorer
)

print("Final model cross-validated scores:", final_cv_scores)
print("Final model mean score:", np.mean(final_cv_scores))

Fitting 5 folds for each of 360 candidates, totalling 1800 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 40}
Best cross-validated score: -5.11271437662143
Final model cross-validated scores: [-5.2917081  -5.03182644 -5.2418707  -5.07788382 -5.34223057]
Final model mean score: -5.197103924917796


In [21]:
# sanity check baseline
print(mean_squared_error(np.mean(anx_phq)*np.ones_like(anx_phq), anx_phq))

2.502264053736481


In [22]:
# no bfi anxiety phq
rf = RandomForestRegressor()
param_grid = {
    'n_estimators': [5, 10, 15, 20, 25, 30, 35, 40],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=mse_scorer,
    cv=kf,
    verbose=1,
    n_jobs=-1  
)
grid_search.fit(new_merged_data_no_bfi_anx_phq, anx_phq)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated score:", grid_search.best_score_)

best_params = grid_search.best_params_
final_rf_model = RandomForestRegressor(**best_params)
final_rf_model.fit(new_merged_data_no_bfi_anx_phq, anx_phq)
final_cv_scores = cross_val_score(
    final_rf_model, 
    new_merged_data_no_bfi_anx_phq, 
    anx_phq, 
    cv=kf, 
    scoring=mse_scorer
)

print("Final model cross-validated scores:", final_cv_scores)
print("Final model mean score:", np.mean(final_cv_scores))

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


/Users/aaron/Desktop/HW/MED_264/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 40}
Best cross-validated score: -1.9985427346490123
Final model cross-validated scores: [-2.07369905 -1.90923651 -2.08576452 -1.93413617 -2.1118258 ]
Final model mean score: -2.022932408703638


In [23]:
# with bfi anxiety phq
rf = RandomForestRegressor()
param_grid = {
    'n_estimators': [5, 10, 15, 20, 25, 30, 35, 40],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=mse_scorer,
    cv=kf,
    verbose=1,
    n_jobs=-1  
)
grid_search.fit(new_merged_data_anx_phq, anx_phq)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated score:", grid_search.best_score_)

best_params = grid_search.best_params_
final_rf_model = RandomForestRegressor(**best_params)
final_rf_model.fit(new_merged_data_anx_phq, anx_phq)
final_cv_scores = cross_val_score(
    final_rf_model, 
    new_merged_data_anx_phq, 
    anx_phq, 
    cv=kf, 
    scoring=mse_scorer
)

print("Final model cross-validated scores:", final_cv_scores)
print("Final model mean score:", np.mean(final_cv_scores))

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


/Users/aaron/Desktop/HW/MED_264/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 35}
Best cross-validated score: -1.6733670485369152
Final model cross-validated scores: [-1.69486163 -1.5682178  -1.74492886 -1.63427173 -1.76870183]
Final model mean score: -1.682196369304494


In [24]:
# sanity check baseline
print(mean_squared_error(np.mean(dep_phq)*np.ones_like(dep_phq), dep_phq))

2.3617106048697485


In [25]:
# no bfi depression phq
rf = RandomForestRegressor()
param_grid = {
    'n_estimators': [5, 10, 15, 20, 25, 30, 35, 40],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=mse_scorer,
    cv=kf,
    verbose=1,
    n_jobs=-1  
)
grid_search.fit(new_merged_data_no_bfi_dep_phq, dep_phq)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated score:", grid_search.best_score_)

best_params = grid_search.best_params_
final_rf_model = RandomForestRegressor(**best_params)
final_rf_model.fit(new_merged_data_no_bfi_dep_phq, dep_phq)
final_cv_scores = cross_val_score(
    final_rf_model, 
    new_merged_data_no_bfi_dep_phq, 
    dep_phq, 
    cv=kf, 
    scoring=mse_scorer
)

print("Final model cross-validated scores:", final_cv_scores)
print("Final model mean score:", np.mean(final_cv_scores))

Fitting 5 folds for each of 360 candidates, totalling 1800 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 40}
Best cross-validated score: -1.8834165151902051
Final model cross-validated scores: [-1.89697259 -1.98133544 -1.80325088 -1.95258393 -1.87193234]
Final model mean score: -1.9012150355003965


In [26]:
# with bfi depression phq
rf = RandomForestRegressor()
param_grid = {
    'n_estimators': [5, 10, 15, 20, 25, 30, 35, 40],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=mse_scorer,
    cv=kf,
    verbose=1,
    n_jobs=-1  
)
grid_search.fit(new_merged_data_dep_phq, dep_phq)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated score:", grid_search.best_score_)

best_params = grid_search.best_params_
final_rf_model = RandomForestRegressor(**best_params)
final_rf_model.fit(new_merged_data_dep_phq, dep_phq)
final_cv_scores = cross_val_score(
    final_rf_model, 
    new_merged_data_dep_phq, 
    dep_phq, 
    cv=kf, 
    scoring=mse_scorer
)

print("Final model cross-validated scores:", final_cv_scores)
print("Final model mean score:", np.mean(final_cv_scores))

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


/Users/aaron/Desktop/HW/MED_264/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 40}
Best cross-validated score: -1.555006446985851
Final model cross-validated scores: [-1.56840288 -1.61559722 -1.50925042 -1.55753269 -1.5697869 ]
Final model mean score: -1.564114020500839
